### Naive LSTM(stateless) with Variable-Length Input to One-Char Output

In [1]:
# importing required packages

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(7)

In [2]:
# define the raw dataset

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [3]:
# create mapping of characters to integers (0-25) and the reverse 

char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

#### Define a maximum input sequence length and set it to a small value like 5 to speed up training. This defines the maximum length of subsequences of the alphabet will be drawn for training. In extensions, this could just as set to the full alphabet (26) or longer if we allow looping back to the start of the sequence.

#### We also need to define the number of random sequences to create, in this case 1000. This too could be more or less. I expect less patterns are actually required.

In [4]:
# prepare the dataset of input to output pairs encoded as integers

num_inputs = 1000
max_len = 5
dataX = []
dataY = []

for i in range(num_inputs):
    start = np.random.randint(len(alphabet)-2)
    end = np.random.randint(start, min(start + max_len, len(alphabet) - 1))
    sequence_in = alphabet[start : end + 1]
    sequence_out = alphabet[end + 1]
    
    dataX.append([char_to_int[char] for char in sequence_in])
    dataY.append(char_to_int[sequence_out])
    print(sequence_in, '->', sequence_out)

PQRST -> U
W -> X
O -> P
OPQ -> R
IJKLM -> N
QRSTU -> V
ABCD -> E
X -> Y
GHIJ -> K
M -> N
XY -> Z
QRST -> U
ABC -> D
JKLMN -> O
OP -> Q
XY -> Z
D -> E
T -> U
B -> C
QRSTU -> V
HIJ -> K
JKLM -> N
ABCDE -> F
X -> Y
V -> W
DE -> F
DEFG -> H
BCDE -> F
EFGH -> I
BCDE -> F
FG -> H
RST -> U
TUV -> W
STUV -> W
LMN -> O
P -> Q
MNOP -> Q
JK -> L
MNOP -> Q
OPQRS -> T
UVWXY -> Z
PQRS -> T
D -> E
EFGH -> I
IJK -> L
WX -> Y
STUV -> W
MNOPQ -> R
P -> Q
WXY -> Z
VWX -> Y
V -> W
HI -> J
KLMNO -> P
UV -> W
JKL -> M
ABCDE -> F
WXY -> Z
M -> N
CDEF -> G
KLMNO -> P
RST -> U
RS -> T
W -> X
J -> K
WX -> Y
JKLMN -> O
MN -> O
L -> M
BCDE -> F
TU -> V
MNOPQ -> R
NOPQR -> S
HIJ -> K
JKLM -> N
STUVW -> X
QRST -> U
N -> O
VWXY -> Z
B -> C
UVWX -> Y
OP -> Q
K -> L
C -> D
X -> Y
ST -> U
JKLM -> N
B -> C
QR -> S
RS -> T
VWXY -> Z
S -> T
NOP -> Q
KLMNO -> P
IJ -> K
EF -> G
MNOP -> Q
WXY -> Z
HI -> J
P -> Q
STUVW -> X
Q -> R
MN -> O
O -> P
C -> D
L -> M
JKLM -> N
K -> L
IJKLM -> N
FGHIJ -> K
LM -> N
OPQ -> R
U -> V
HIJ

In [8]:
# convert list of lists to array and pad sequences if needed

X = sequence.pad_sequences(dataX, maxlen=max_len,dtype='float32')
print('X :', X[0:5]) # printing first 5 values

X : [[15. 16. 17. 18. 19.]
 [ 0.  0.  0.  0. 22.]
 [ 0.  0.  0.  0. 14.]
 [ 0.  0. 14. 15. 16.]
 [ 8.  9. 10. 11. 12.]]


In [10]:
# reshape X to be [samples, time steps, features]

X = np.reshape(X, (X.shape[0], max_len, 1))
print('X :', X[0:5])# printing first 5 values

X : [[[15.]
  [16.]
  [17.]
  [18.]
  [19.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [22.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [14.]]

 [[ 0.]
  [ 0.]
  [14.]
  [15.]
  [16.]]

 [[ 8.]
  [ 9.]
  [10.]
  [11.]
  [12.]]]


In [11]:
# normalize the input integers to the range 0-to-1, the range of
# the sigmoid activation functions used by the LSTM network

X = X/float(len(alphabet))
print('X :', X[0:5]) # printing first  5 values

X : [[[0.5769231 ]
  [0.61538464]
  [0.65384614]
  [0.6923077 ]
  [0.7307692 ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.84615386]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.53846157]]

 [[0.        ]
  [0.        ]
  [0.53846157]
  [0.5769231 ]
  [0.61538464]]

 [[0.30769232]
  [0.34615386]
  [0.3846154 ]
  [0.42307693]
  [0.46153846]]]


In [12]:
# One hot encode the output variables

y = utils.to_categorical(dataY)
print(' Encoded y :', y[0:5])# printing first 5 values

 Encoded y : [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [13]:
# creating and naive LSTM stateless model

model = Sequential()
model.add(LSTM(units=32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Keras shuffles the training dataset before each training epoch. To ensure the training data patterns remain sequential, we can disable this shuffling.

import datetime
start = datetime.datetime.now()

print(model.summary())
batch_size=1
model.fit(X, y, epochs=500, batch_size=1, verbose=1, shuffle=True)


stop = datetime.datetime.now()
print("Total time taken : ", str(stop - start))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 26)                858       
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
1000/1000 [==============================] - 9s 5ms/step - loss: 3.0833 - accuracy: 0.0850
Epoch 2/500
1000/1000 [==============================] - 5s 5ms/step - loss: 2.7986 - accuracy: 0.1330
Epoch 3/500
1000/1000 [==============================] - 5s 5ms/step - loss: 2.4755 - accuracy: 0.1860
Epoch 4/500
1000/1000 [==============================] - 5s 5ms/step - loss: 2.2518 - accuracy: 0.2390
Epoch 5/500
1000/1000 [==============================] - 5s 5ms/step - loss: 2.0869 - accuracy: 0.2

1000/1000 [==============================] - 5s 5ms/step - loss: 0.4786 - accuracy: 0.8460
Epoch 74/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5023 - accuracy: 0.8230
Epoch 75/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4746 - accuracy: 0.8490
Epoch 76/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4835 - accuracy: 0.8330
Epoch 77/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4571 - accuracy: 0.8570
Epoch 78/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4978 - accuracy: 0.8310
Epoch 79/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4506 - accuracy: 0.8530
Epoch 80/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4510 - accuracy: 0.8490
Epoch 81/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4443 - accuracy: 0.8470
Epoch 82/500
1000/1000 [==============================] - 5s 5ms/step - loss:

1000/1000 [==============================] - 5s 5ms/step - loss: 0.2991 - accuracy: 0.9100
Epoch 152/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3775 - accuracy: 0.8850
Epoch 153/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2786 - accuracy: 0.9150
Epoch 154/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2831 - accuracy: 0.9210
Epoch 155/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2892 - accuracy: 0.9160
Epoch 156/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2891 - accuracy: 0.9100
Epoch 157/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3683 - accuracy: 0.8820
Epoch 158/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3148 - accuracy: 0.9060
Epoch 159/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3069 - accuracy: 0.9030
Epoch 160/500
1000/1000 [==============================] - 5s 5ms/ste

1000/1000 [==============================] - 5s 5ms/step - loss: 0.2175 - accuracy: 0.9430
Epoch 230/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2085 - accuracy: 0.9420
Epoch 231/500
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2100 - accuracy: 0.9380
Epoch 232/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2152 - accuracy: 0.9260
Epoch 233/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3449 - accuracy: 0.8960
Epoch 234/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1996 - accuracy: 0.9370
Epoch 235/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2022 - accuracy: 0.9390
Epoch 236/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2103 - accuracy: 0.9320
Epoch 237/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2101 - accuracy: 0.9340
Epoch 238/500
1000/1000 [==============================] - 5s 5ms/ste

1000/1000 [==============================] - 5s 5ms/step - loss: 0.1659 - accuracy: 0.9480
Epoch 308/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1691 - accuracy: 0.9490
Epoch 309/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1635 - accuracy: 0.9540
Epoch 310/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2162 - accuracy: 0.9400
Epoch 311/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1612 - accuracy: 0.9500
Epoch 312/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1648 - accuracy: 0.9510
Epoch 313/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1643 - accuracy: 0.9500
Epoch 314/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2368 - accuracy: 0.9330
Epoch 315/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1536 - accuracy: 0.9590
Epoch 316/500
1000/1000 [==============================] - 5s 5ms/ste

1000/1000 [==============================] - 5s 5ms/step - loss: 0.1323 - accuracy: 0.9640
Epoch 385/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2232 - accuracy: 0.9440
Epoch 386/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1227 - accuracy: 0.9700
Epoch 387/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1251 - accuracy: 0.9660
Epoch 388/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1244 - accuracy: 0.9680
Epoch 389/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1265 - accuracy: 0.9630
Epoch 390/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1300 - accuracy: 0.9610
Epoch 391/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1320 - accuracy: 0.9580
Epoch 392/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1289 - accuracy: 0.9620
Epoch 393/500
1000/1000 [==============================] - 5s 5ms/ste

1000/1000 [==============================] - 5s 5ms/step - loss: 0.1089 - accuracy: 0.9730
Epoch 463/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1066 - accuracy: 0.9720
Epoch 464/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1072 - accuracy: 0.9710
Epoch 465/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1062 - accuracy: 0.9740
Epoch 466/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1398 - accuracy: 0.9680
Epoch 467/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1025 - accuracy: 0.9800
Epoch 468/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1056 - accuracy: 0.9700
Epoch 469/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1028 - accuracy: 0.9700
Epoch 470/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1855 - accuracy: 0.9580
Epoch 471/500
1000/1000 [==============================] - 5s 5ms/ste

In [15]:
# summarize performance of the model

scores = model.evaluate(X, y, verbose=1)
print("Model Accuracy :%.2f%%" % (scores[1]*100))

32/32 [==============================] - 1s 4ms/step - loss: 0.0859 - accuracy: 0.9920
Model Accuracy :99.20%


In [18]:
# demonstrate predicting random patterns
print("Test a Random Pattern :")
for i in range(0, 20):
    pattern_index = np.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = sequence.pad_sequences([pattern], maxlen=max_len, dtype='float32')
    x = np.reshape(x, (1, max_len, 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

Test a Random Pattern :
['E', 'F'] -> G
['K', 'L', 'M'] -> N
['B'] -> C
['C'] -> D
['R', 'S'] -> T
['A', 'B', 'C'] -> D
['C', 'D', 'E'] -> F
['N', 'O', 'P'] -> Q
['C', 'D'] -> E
['L', 'M'] -> N
['F', 'G', 'H', 'I', 'J'] -> K
['N', 'O', 'P', 'Q'] -> R
['C', 'D', 'E', 'F', 'G'] -> H
['A', 'B', 'C'] -> D
['R', 'S', 'T', 'U', 'V'] -> W
['B', 'C', 'D'] -> E
['F', 'G'] -> H
['K'] -> L
['W', 'X'] -> Y
['B'] -> C


### We can see that although the model did not learn the alphabet perfectly from the randomly generated subsequences, it did very well. The model was not tuned and may require more training or a larger network, or both 